In [1]:
import argparse
import os
import random
import shutil
import time
import warnings
import sys
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.utils.model_zoo as model_zoo
import torchvision
import torch.optim.lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable

from PIL import Image

In [2]:
BATCH_SIZE = 20
EPOCH = 50
GAMMA = 0.9
STEP_SIZE = 200
LR = 0.001
USE_GPU = True
decoder = ['buoy', 'dock', 'light_buoy', 'totem']
data_transform = transforms.Compose([
            transforms.Resize(227),
            #transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

In [3]:
list_path = '/media/arg_ws3/TOSHIBA EXT/data/trajectory/root.txt'
img_path = '/media/arg_ws3/TOSHIBA EXT/data/trajectory/images/'
ann_path = '/media/arg_ws3/TOSHIBA EXT/data/trajectory/annotations/'
model_path = '../model/'
if not os.path.exists(model_path):
    os.makedirs(model_path)
data_list_file = open(list_path,'r')
raw_data_list = data_list_file.read().splitlines()
data_list = []
for data in raw_data_list:
    data_split = data.split(',')
    first_frame = data_split[0]
    data_len = int(data_split[1])
    if data_len >= 10:
        data_list.append([first_frame, data_len])

## Define Layer Modules

In [4]:
class LRN(nn.Module):
    def __init__(self, local_size=1, alpha=1.0, beta=0.75, ACROSS_CHANNELS=True):
        super(LRN, self).__init__()
        self.ACROSS_CHANNELS = ACROSS_CHANNELS
        if ACROSS_CHANNELS:
            self.average=nn.AvgPool3d(kernel_size=(local_size, 1, 1),
                    stride=1,
                    padding=(int((local_size-1.0)/2), 0, 0))
        else:
            self.average=nn.AvgPool2d(kernel_size=local_size,
                    stride=1,
                    padding=int((local_size-1.0)/2))
        self.alpha = alpha
        self.beta = beta


    def forward(self, x):
        if self.ACROSS_CHANNELS:
            div = x.pow(2).unsqueeze(1)
            div = self.average(div).squeeze(1)
            div = div.mul(self.alpha).add(1.0).pow(self.beta)
        else:
            div = x.pow(2)
            div = self.average(div)
            div = div.mul(self.alpha).add(1.0).pow(self.beta)
        x = x.div(div)
        return x
    
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class alexnet_conv_layers(nn.Module):
    def __init__(self):
        super(alexnet_conv_layers, self).__init__()
        self.base_features = torchvision.models.alexnet(pretrained = True).features
        self.skip1 = nn.Sequential(
            nn.Conv2d(64, out_channels=16, kernel_size=1, stride=1),
            nn.PReLU(),
            Flatten()
        )
        self.skip2 = nn.Sequential(
            nn.Conv2d(192, out_channels=32, kernel_size=1, stride=1),
            nn.PReLU(),
            Flatten()
        )
        self.skip5 = nn.Sequential(
            nn.Conv2d(256, out_channels=64, kernel_size=1, stride=1),
            nn.PReLU(),
            Flatten()
        )
        self.conv6 = nn.Sequential(
            nn.Linear(37104 * 2, 2048),
            nn.ReLU()
        )
        
        # Freeze those weights
        for p in self.base_features.parameters():
            p.requires_grad = False
            
    def forward(self, x, y):
        layer_extractor_x = []
        layer_extractor_y = []
        for idx, model in enumerate(self.base_features):
            x = model(x)
            y = model(y)
            if idx in {2, 5, 11}: # layer output of conv1, conv2 , conv5(before pooling layer)
                layer_extractor_x.append(x)
                layer_extractor_y.append(y)
                
        x_out_flat = x.view(1, -1) #(1, 256, 6, 6) --> (1, 9216)
        x_out_skip1 = self.skip1(layer_extractor_x[0]) #(1, 64, 27, 27) -> (11664)
        x_out_skip2 = self.skip2(layer_extractor_x[1]) #(1, 192, 13, 13) -> (5408)
        x_out_skip5 = self.skip5(layer_extractor_x[2]) #(1, 256, 13, 13) -> (10816)
        x_out = torch.cat((x_out_skip1, x_out_skip2, x_out_skip5, x_out_flat), dim=1)
        
        y_out_flat = y.view(1, -1) #(1, 256, 6, 6) --> (1, 9216)
        y_out_skip1 = self.skip1(layer_extractor_y[0]) #(1, 64, 27, 27) -> (11664)
        y_out_skip2 = self.skip2(layer_extractor_y[1]) #(1, 192, 13, 13) -> (5408)
        y_out_skip5 = self.skip5(layer_extractor_y[2]) #(1, 256, 13, 13) -> (10816)
        y_out = torch.cat((y_out_skip1, y_out_skip2, y_out_skip5, y_out_flat), dim=1)
        
        final_out = torch.cat((x_out, y_out), dim=1)
        conv_out = self.conv6(final_out) # (1, 2048)
        return conv_out
def adjust_learning_rate(optimizer):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    for param_group in optimizer.param_groups:
        param_group['lr'] = param_group['lr'] * 0.1
        print("Change learning rate to: ", param_group['lr'])

In [5]:
a = Variable(torch.rand(1, 51)).cuda()
print(a.shape)
b = torch.zeros(1, 51).cuda()
print(b.shape)

torch.Size([1, 51])
torch.Size([1, 51])


In [6]:
class PredictNet(nn.Module):
    def __init__(self):
        super(PredictNet,self).__init__()
        self.LSTM_SIZE = 256
        alexnet = torchvision.models.alexnet(pretrained = True)
        self.feature = alexnet.features
        classifier = list(alexnet.classifier.children())
        self.classifier = nn.Sequential(*classifier[:-1])
        self.lstm1 =nn.LSTMCell(4096 + 2, self.LSTM_SIZE)
        self.lstm2 =nn.LSTMCell(self.LSTM_SIZE, self.LSTM_SIZE)
        
        self.lstm1_x =nn.LSTMCell(4096 + 1, self.LSTM_SIZE)
        self.lstm2_x = nn.LSTMCell(self.LSTM_SIZE, self.LSTM_SIZE)
        self.lstm1_y =nn.LSTMCell(4096 + 1, self.LSTM_SIZE)
        self.lstm2_y = nn.LSTMCell(self.LSTM_SIZE, self.LSTM_SIZE)
        self.linear_x = nn.Linear(self.LSTM_SIZE, 1)
        self.linear_y = nn.Linear(self.LSTM_SIZE, 1)
        self.linear = nn.Linear(self.LSTM_SIZE, 2)
        
        self.h_1 = self.get_hidden(self.LSTM_SIZE)
        self.h_2 = self.get_hidden(self.LSTM_SIZE)
        self.c_1 = self.get_hidden(self.LSTM_SIZE)
        self.c_2 = self.get_hidden(self.LSTM_SIZE)
        
        self.h_x1 = self.get_hidden(self.LSTM_SIZE)
        self.h_y1 = self.get_hidden(self.LSTM_SIZE)
        self.h_x2 = self.get_hidden(self.LSTM_SIZE)
        self.h_y2 = self.get_hidden(self.LSTM_SIZE)
        self.c_x1 = self.get_hidden(self.LSTM_SIZE)
        self.c_y1 = self.get_hidden(self.LSTM_SIZE)
        self.c_x2 = self.get_hidden(self.LSTM_SIZE)
        self.c_y2 = self.get_hidden(self.LSTM_SIZE)

    def init_hidden(self, num):
        self.h_1 = self.get_hidden(num)
        self.h_2 = self.get_hidden(num)
        self.c_1 = self.get_hidden(num)
        self.c_2 = self.get_hidden(num)
        
        self.h_x1 = self.get_hidden(num)
        self.h_y1 = self.get_hidden(num)
        self.h_x2 = self.get_hidden(num)
        self.h_y2 = self.get_hidden(num)
        self.c_x1 = self.get_hidden(num)
        self.c_y1 = self.get_hidden(num)
        self.c_x2 = self.get_hidden(num)
        self.c_y2 = self.get_hidden(num)

    def get_hidden(self, num):
        if USE_GPU:
            #return (Variable(torch.rand(1, num)).cuda(), Variable(torch.rand(1, num)).cuda())
            return Variable(torch.rand(1, num)).cuda()
            #return torch.zeros(1, num).cuda()
        else:
            #return (Variable(torch.rand(1, num)), Variable(torch.rand(1, num)))
            return Variable(torch.rand(1, num))
            #return torch.zeros(t1, num)
        
    def forward(self, img, input_x, input_y):
        '''h_x1 = torch.zeros(1, self.LSTM_SIZE).cuda()
        h_y1 = torch.zeros(1, self.LSTM_SIZE).cuda()
        h_x2 = torch.zeros(1, self.LSTM_SIZE).cuda()
        h_y2 = torch.zeros(1, self.LSTM_SIZE).cuda()
        c_x1 = torch.zeros(1, self.LSTM_SIZE).cuda()
        c_y1 = torch.zeros(1, self.LSTM_SIZE).cuda()
        c_x2 = torch.zeros(1, self.LSTM_SIZE).cuda()
        c_y2 = torch.zeros(1, self.LSTM_SIZE).cuda()'''
        
        img_features = self.feature(img.unsqueeze(0))
        img_features = img_features.view(img_features.size(0), -1)
        img_features = self.classifier(img_features).view(-1)
        x = input_x
        y = input_y
        
        '''cat_x = torch.cat((img_features, x), dim=0).view(1, -1)
        h_x1, c_x1 = self.lstm1_x(cat_x, (h_x1, c_x1))
        h_x2, c_x2 = self.lstm2_y(h_x1, (h_x2, c_x2))
        output_x = self.linear_x(h_x2)
        
        cat_y = torch.cat((img_features, y), dim=0).view(1, -1)
        h_y1, c_y1 = self.lstm1_y(cat_y, (h_y1, c_y1))
        h_y2, c_y2 = self.lstm2_y(h_y1, (h_y2, c_y2))
        output_y = self.linear_y(h_y2)'''
        
        '''cat_x = torch.cat((img_features, x), dim=0).view(1, -1)
        self.h_x1, self.c_x1 = self.lstm1_x(cat_x, (self.h_x1, self.c_x1))
        self.h_x2, self.c_x2 = self.lstm2_y(self.h_x1, (self.h_x2, self.c_x2))
        output_x = self.linear_x(self.h_x2)
        
        cat_y = torch.cat((img_features, y), dim=0).view(1, -1)
        self.h_y1, self.c_y1 = self.lstm1_y(cat_y, (self.h_y1, self.c_y1))
        self.h_y2, self.c_y2 = self.lstm2_y(self.h_y1, (self.h_y2, self.c_y2))
        output_y = self.linear_y(self.h_y2)
        output = torch.cat((output_x, output_y))'''
        
        cat = torch.cat((img_features, x, y), dim=0).view(1, -1)
        self.h_1, self.c_1 = self.lstm1(cat, (self.h_1, self.c_1))
        self.h_2, self.c_2 = self.lstm2(self.h_1, (self.h_2, self.c_2))
        output = self.linear(self.h_2)
        
        return output

In [7]:
'''normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
img_transform = transforms.Compose([
                    transforms.RandomResizedCrop(224),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    normalize,
                    ])'''
def train_model(optimizer, criterion, net, num_epochs):
    x = None
    y = None
    img = None
    for epoch in range(num_epochs):
        if epoch != 0 and epoch % 5 == 0:
            adjust_learning_rate(optim)
        curr_loss = 0.0
        dataset_size = len(data_list)
        sample_list = random.sample(range(0, dataset_size), dataset_size)
        for idx in sample_list:
            frame_name = data_list[idx][0]
            data_len = data_list[idx][1]
            target_x = []
            target_y = []
            target = []
            out_x = []
            out_y = []
            out = []
            net.init_hidden(net.LSTM_SIZE)
            for frame_num in range(data_len-1):
                file = open(ann_path + frame_name + '.txt','r')
                file_split = file.read().splitlines()
                next_frame = file_split[1]
                img = Image.open(img_path + next_frame + '.jpg')
                img = img.convert('RGB')
                img = img_transform(img)
                x = torch.tensor([float(file_split[4]) - float(file_split[2])], requires_grad=False)
                y = torch.tensor([float(file_split[5]) - float(file_split[3])], requires_grad=False)
                if USE_GPU:
                    img = img.cuda()
                    x = x.cuda()
                    y = y.cuda()
                if frame_num != 0:
                    target_x = target_x + [x]
                    target_y = target_y + [y]
                output_x, output_y = net(img, x, y)
                out_x = out_x + [output_x]
                out_y = out_y + [output_y]
                frame_name = next_frame
            file = open(ann_path + frame_name + '.txt','r')
            file_split = file.read().splitlines()
            x = torch.tensor([float(file_split[4]) - float(file_split[2])])
            y = torch.tensor([float(file_split[5]) - float(file_split[3])])
            if USE_GPU:
                x = x.cuda()
                y = y.cuda()
            target_x = target_x + [x]
            target_y = target_y + [y]
                
            target_x = torch.stack(target_x, 1).squeeze(1).view(-1)
            target_y = torch.stack(target_y, 1).squeeze(1).view(-1)
            target = torch.cat((target_x, target_y))
            out_x = torch.stack(out_x, 1).squeeze(2).view(-1)
            out_y = torch.stack(out_y, 1).squeeze(2).view(-1)
            out = torch.cat((out_x, out_y)) #torch.Size([n])
            
            def clousure():
                optimizer.zero_grad()
                loss = criterion(out, target)
                print(epoch, '>>', idx, '>> loss:', loss.item())
                loss.backward(retain_graph = True)
                return loss
            optimizer.step(clousure)
            #torch.cuda.empty_cache()
            
    return net

'''np.random.seed(0)
torch.manual_seed(0)
net = PredictNet()
#net.double()
criterion = nn.MSELoss()
optimizer = torch.optim.LBFGS(net.parameters(), lr=0.1)
if USE_GPU:
    net = net.cuda()
    criterion = criterion.cuda()
net = train_model(optimizer, criterion, net, 1)'''

'np.random.seed(0)\ntorch.manual_seed(0)\nnet = PredictNet()\n#net.double()\ncriterion = nn.MSELoss()\noptimizer = torch.optim.LBFGS(net.parameters(), lr=0.1)\nif USE_GPU:\n    net = net.cuda()\n    criterion = criterion.cuda()\nnet = train_model(optimizer, criterion, net, 1)'

In [8]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
img_transform = transforms.Compose([
                    transforms.RandomResizedCrop(224),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    normalize,
                    ])
def train_model(optimizer, criterion, net, num_epochs):
    x = None
    y = None
    img = None
    epoch_loss = 0.
    for epoch in range(num_epochs):
        
        
        curr_loss = 0.0
        dataset_size = len(data_list)
        sample_list = random.sample(range(0, dataset_size), dataset_size)
        if epoch != 0 and epoch % 30 == 0:
            adjust_learning_rate(optimizer)
        if epoch != 0 and epoch % 5 == 0:
            torch.save(net.state_dict(), model_path + 'predict_' + str(epoch) + '_' + str(epoch_loss/dataset_size) + '.pth')
        epoch_loss = 0.
        for i, idx in enumerate(sample_list):
            frame_name = data_list[idx][0]
            data_len = data_list[idx][1]
            net.init_hidden(net.LSTM_SIZE)
            total_loss = 0.
            for frame_num in range(data_len-1):
                file = open(ann_path + frame_name + '.txt','r')
                file_split = file.read().splitlines()
                next_frame = file_split[1]
                img = Image.open(img_path + next_frame + '.jpg')
                img = img.convert('RGB')
                img = img_transform(img)
                x = torch.tensor([(float(file_split[4]) - float(file_split[2]))*10])
                y = torch.tensor([(float(file_split[5]) - float(file_split[3]))*10])
                if USE_GPU:
                    img = img.cuda()
                    x = x.cuda()
                    y = y.cuda()
                output = net(img, x, y)
                frame_name = next_frame
                file = open(ann_path + frame_name + '.txt','r')
                file_split = file.read().splitlines()
                target_x = torch.tensor([(float(file_split[4]) - float(file_split[2]))*10])
                target_y = torch.tensor([(float(file_split[4]) - float(file_split[2]))*10])
                if USE_GPU:
                    target_x = target_x.cuda()
                    target_y = target_y.cuda()
                target = torch.cat((target_x, target_y))
                loss = criterion(output, target)
                total_loss = total_loss + loss.item()
                optimizer.zero_grad()
                loss.backward(retain_graph=True)
                optimizer.step()
            epoch_loss = epoch_loss + total_loss/(data_len-1)
            if i % 10 == 0:
                print('Epoch: ', epoch, '|| Seq: ' + str(i) + ' / '+ str(dataset_size), '|| loss:', total_loss/(data_len-1))
        print('Epoch: ', epoch, '|| Loss: ', (epoch_loss/dataset_size))
    return net

np.random.seed(0)
torch.manual_seed(0)
net = PredictNet()
#net.load_state_dict(torch.load("/home/arg_ws3/argbot/catkin_ws/src/network/model/predict_5_0.0.pth"))
#net.double()
criterion = nn.MSELoss()
optimizer = torch.optim.RMSprop(net.parameters(), lr=0.001)
if USE_GPU:
    net = net.cuda()
    criterion = criterion.cuda()
net = train_model(optimizer, criterion, net, 201)

Epoch:  0 || Seq: 0 / 172 || loss: 90.99676804921843
Epoch:  0 || Seq: 10 / 172 || loss: 100.79533333910837
Epoch:  0 || Seq: 20 / 172 || loss: 196.01861368870595
Epoch:  0 || Seq: 30 / 172 || loss: 298.86436388227673
Epoch:  0 || Seq: 40 / 172 || loss: 25.858854489934554
Epoch:  0 || Seq: 50 / 172 || loss: 49.74922990558609
Epoch:  0 || Seq: 60 / 172 || loss: 4.43985784612596
Epoch:  0 || Seq: 70 / 172 || loss: 83.84747210106191
Epoch:  0 || Seq: 80 / 172 || loss: 23.133172557046194
Epoch:  0 || Seq: 90 / 172 || loss: 172.89617608470672
Epoch:  0 || Seq: 100 / 172 || loss: 10.111371204257011
Epoch:  0 || Seq: 110 / 172 || loss: 181.9653766155243
Epoch:  0 || Seq: 120 / 172 || loss: 113.53533525157557
Epoch:  0 || Seq: 130 / 172 || loss: 61.13784436980883
Epoch:  0 || Seq: 140 / 172 || loss: 305.86746164917946
Epoch:  0 || Seq: 150 / 172 || loss: 102.12728127360384
Epoch:  0 || Seq: 160 / 172 || loss: 2.8743587280623615
Epoch:  0 || Seq: 170 / 172 || loss: 218.61271546103737
Epoch:  0 

Epoch:  8 || Seq: 0 / 172 || loss: 3.988186791539192
Epoch:  8 || Seq: 10 / 172 || loss: 156.5586829481735
Epoch:  8 || Seq: 20 / 172 || loss: 7.289586374070495
Epoch:  8 || Seq: 30 / 172 || loss: 181.73599937646404
Epoch:  8 || Seq: 40 / 172 || loss: 3.523729741461575
Epoch:  8 || Seq: 50 / 172 || loss: 44.92291262894869
Epoch:  8 || Seq: 60 / 172 || loss: 29.4814059470143
Epoch:  8 || Seq: 70 / 172 || loss: 328.5134985588491
Epoch:  8 || Seq: 80 / 172 || loss: 7.805354603699276
Epoch:  8 || Seq: 90 / 172 || loss: 35.28450381522998
Epoch:  8 || Seq: 100 / 172 || loss: 2.280439412944313
Epoch:  8 || Seq: 110 / 172 || loss: 4.6612158318325365
Epoch:  8 || Seq: 120 / 172 || loss: 21.09280849248171
Epoch:  8 || Seq: 130 / 172 || loss: 26.520316538930636
Epoch:  8 || Seq: 140 / 172 || loss: 10.449601402428621
Epoch:  8 || Seq: 150 / 172 || loss: 129.79698225872025
Epoch:  8 || Seq: 160 / 172 || loss: 9.29745793600367
Epoch:  8 || Seq: 170 / 172 || loss: 18.25991296126846
Epoch:  8 || Loss:

Epoch:  15 || Seq: 170 / 172 || loss: 7.205359734689409
Epoch:  15 || Loss:  61.215717663120884
Epoch:  16 || Seq: 0 / 172 || loss: 9.8794913017191
Epoch:  16 || Seq: 10 / 172 || loss: 43.98685060370092
Epoch:  16 || Seq: 20 / 172 || loss: 13.753465568232867
Epoch:  16 || Seq: 30 / 172 || loss: 20.90573579427742
Epoch:  16 || Seq: 40 / 172 || loss: 160.45187806618364
Epoch:  16 || Seq: 50 / 172 || loss: 175.06846104167403
Epoch:  16 || Seq: 60 / 172 || loss: 15.05189286065778
Epoch:  16 || Seq: 70 / 172 || loss: 61.67074175179005
Epoch:  16 || Seq: 80 / 172 || loss: 9.520323563612214
Epoch:  16 || Seq: 90 / 172 || loss: 12.95519976483451
Epoch:  16 || Seq: 100 / 172 || loss: 23.188692173399474
Epoch:  16 || Seq: 110 / 172 || loss: 0.8558970311373149
Epoch:  16 || Seq: 120 / 172 || loss: 64.25232198244582
Epoch:  16 || Seq: 130 / 172 || loss: 15.916577694447417
Epoch:  16 || Seq: 140 / 172 || loss: 36.67743196165189
Epoch:  16 || Seq: 150 / 172 || loss: 186.4296432348589
Epoch:  16 || S

Epoch:  23 || Seq: 150 / 172 || loss: 1.7783598009246397
Epoch:  23 || Seq: 160 / 172 || loss: 23.94622819514378
Epoch:  23 || Seq: 170 / 172 || loss: 2.376844108514524
Epoch:  23 || Loss:  65.61357518323479
Epoch:  24 || Seq: 0 / 172 || loss: 1.2994023669469992
Epoch:  24 || Seq: 10 / 172 || loss: 4.761895168601768
Epoch:  24 || Seq: 20 / 172 || loss: 31.33940552002213
Epoch:  24 || Seq: 30 / 172 || loss: 2.403219158880765
Epoch:  24 || Seq: 40 / 172 || loss: 142.86068184840042
Epoch:  24 || Seq: 50 / 172 || loss: 2.8550958052219357
Epoch:  24 || Seq: 60 / 172 || loss: 119.54953050647269
Epoch:  24 || Seq: 70 / 172 || loss: 186.07500284910202
Epoch:  24 || Seq: 80 / 172 || loss: 47.632067817908066
Epoch:  24 || Seq: 90 / 172 || loss: 11.144619779894128
Epoch:  24 || Seq: 100 / 172 || loss: 73.73626357865093
Epoch:  24 || Seq: 110 / 172 || loss: 19.46833936889705
Epoch:  24 || Seq: 120 / 172 || loss: 46.398866674154995
Epoch:  24 || Seq: 130 / 172 || loss: 21.811099308232468
Epoch:  24

Epoch:  31 || Seq: 120 / 172 || loss: 95.59677578852727
Epoch:  31 || Seq: 130 / 172 || loss: 269.45021842916805
Epoch:  31 || Seq: 140 / 172 || loss: 108.71424359083176
Epoch:  31 || Seq: 150 / 172 || loss: 17.877661092281343
Epoch:  31 || Seq: 160 / 172 || loss: 252.28568444532507
Epoch:  31 || Seq: 170 / 172 || loss: 310.4108480724196
Epoch:  31 || Loss:  128.70066979076483
Epoch:  32 || Seq: 0 / 172 || loss: 463.13542646376624
Epoch:  32 || Seq: 10 / 172 || loss: 197.18602628077497
Epoch:  32 || Seq: 20 / 172 || loss: 5.080745633544498
Epoch:  32 || Seq: 30 / 172 || loss: 106.14952096730471
Epoch:  32 || Seq: 40 / 172 || loss: 138.83884070813656
Epoch:  32 || Seq: 50 / 172 || loss: 187.35451565867987
Epoch:  32 || Seq: 60 / 172 || loss: 86.3153649220864
Epoch:  32 || Seq: 70 / 172 || loss: 6.017449736595154
Epoch:  32 || Seq: 80 / 172 || loss: 228.3295732246091
Epoch:  32 || Seq: 90 / 172 || loss: 129.06849628145045
Epoch:  32 || Seq: 100 / 172 || loss: 201.7995296198389
Epoch:  32

...

Epoch:  110 || Seq: 30 / 172 || loss: 8.146857768297195
Epoch:  110 || Seq: 40 / 172 || loss: 279.8096141199912
Epoch:  110 || Seq: 50 / 172 || loss: 115.94602421909804
Epoch:  110 || Seq: 60 / 172 || loss: 206.27910410563152
Epoch:  110 || Seq: 70 / 172 || loss: 45.81143355369568
Epoch:  110 || Seq: 80 / 172 || loss: 50.18365586371649
Epoch:  110 || Seq: 90 / 172 || loss: 3.7767375239024035
Epoch:  110 || Seq: 100 / 172 || loss: 75.07371783256531
Epoch:  110 || Seq: 110 / 172 || loss: 47.401743221282956
Epoch:  110 || Seq: 120 / 172 || loss: 86.88957009996686
Epoch:  110 || Seq: 130 / 172 || loss: 211.65007963634673
Epoch:  110 || Seq: 140 / 172 || loss: 154.4742317199707
Epoch:  110 || Seq: 150 / 172 || loss: 21.12568686803182
Epoch:  110 || Seq: 160 / 172 || loss: 254.1183586854201
Epoch:  110 || Seq: 170 / 172 || loss: 98.496033008282
Epoch:  110 || Loss:  167.61334827741686
Epoch:  111 || Seq: 0 / 172 || loss: 256.45363896687826
Epoch:  111 || Seq: 10 / 172 || loss: 403.5669916226

Epoch:  117 || Seq: 170 / 172 || loss: 71.85996748850896
Epoch:  117 || Loss:  170.55130921935609
Epoch:  118 || Seq: 0 / 172 || loss: 57.59023582935333
Epoch:  118 || Seq: 10 / 172 || loss: 117.78181985708383
Epoch:  118 || Seq: 20 / 172 || loss: 101.89977149168651
Epoch:  118 || Seq: 30 / 172 || loss: 401.79774375583816
Epoch:  118 || Seq: 40 / 172 || loss: 416.0406232561384
Epoch:  118 || Seq: 50 / 172 || loss: 269.3784860309802
Epoch:  118 || Seq: 60 / 172 || loss: 7.394460752484433
Epoch:  118 || Seq: 70 / 172 || loss: 242.72089297771453
Epoch:  118 || Seq: 80 / 172 || loss: 143.51511588463416
Epoch:  118 || Seq: 90 / 172 || loss: 42.471140480041505
Epoch:  118 || Seq: 100 / 172 || loss: 27.549252250364848
Epoch:  118 || Seq: 110 / 172 || loss: 386.0720491409302
Epoch:  118 || Seq: 120 / 172 || loss: 150.39960405561658
Epoch:  118 || Seq: 130 / 172 || loss: 10.854068623412223
Epoch:  118 || Seq: 140 / 172 || loss: 154.8272507455614
Epoch:  118 || Seq: 150 / 172 || loss: 28.3924539

Epoch:  125 || Seq: 110 / 172 || loss: 8.921585619449615
Epoch:  125 || Seq: 120 / 172 || loss: 139.84544471002394
Epoch:  125 || Seq: 130 / 172 || loss: 26.359266155958174
Epoch:  125 || Seq: 140 / 172 || loss: 111.60177128890464
Epoch:  125 || Seq: 150 / 172 || loss: 242.33567449781629
Epoch:  125 || Seq: 160 / 172 || loss: 346.82707336970736
Epoch:  125 || Seq: 170 / 172 || loss: 28.94562043641743
Epoch:  125 || Loss:  168.4312982619538
Epoch:  126 || Seq: 0 / 172 || loss: 453.6405053138733
Epoch:  126 || Seq: 10 / 172 || loss: 295.54597695668537
Epoch:  126 || Seq: 20 / 172 || loss: 12.165681856019157
Epoch:  126 || Seq: 30 / 172 || loss: 8.553824070375413
Epoch:  126 || Seq: 40 / 172 || loss: 3.6794644698108496
Epoch:  126 || Seq: 50 / 172 || loss: 327.0911602973938
Epoch:  126 || Seq: 60 / 172 || loss: 246.32068753242493
Epoch:  126 || Seq: 70 / 172 || loss: 295.5489911528734
Epoch:  126 || Seq: 80 / 172 || loss: 6.6758823694013385
Epoch:  126 || Seq: 90 / 172 || loss: 107.169169

Epoch:  133 || Seq: 60 / 172 || loss: 144.81119713416467
Epoch:  133 || Seq: 70 / 172 || loss: 297.2934920403265
Epoch:  133 || Seq: 80 / 172 || loss: 189.1466638124906
Epoch:  133 || Seq: 90 / 172 || loss: 40.96249241204489
Epoch:  133 || Seq: 100 / 172 || loss: 276.46886751386853
Epoch:  133 || Seq: 110 / 172 || loss: 475.11264979839325
Epoch:  133 || Seq: 120 / 172 || loss: 239.30413754781088
Epoch:  133 || Seq: 130 / 172 || loss: 63.2588465841193
Epoch:  133 || Seq: 140 / 172 || loss: 14.542454541170013
Epoch:  133 || Seq: 150 / 172 || loss: 112.13514796158542
Epoch:  133 || Seq: 160 / 172 || loss: 238.66267502307892
Epoch:  133 || Seq: 170 / 172 || loss: 6.309310382062739
Epoch:  133 || Loss:  177.87316671967938
Epoch:  134 || Seq: 0 / 172 || loss: 101.41891489530865
Epoch:  134 || Seq: 10 / 172 || loss: 312.7892436981201
Epoch:  134 || Seq: 20 / 172 || loss: 208.06153263648352
Epoch:  134 || Seq: 30 / 172 || loss: 194.27597671084933
Epoch:  134 || Seq: 40 / 172 || loss: 3.3961217

KeyboardInterrupt: 